# [Deep Learning](https://www.cc.gatech.edu/~hays/compvision/proj6/)

## Setup

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import cv2
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
import os.path as osp
import matplotlib.pyplot as plt
from utils import *
import student_code as sc
from torchvision.models import alexnet
import timeit

data_path = osp.join('../data', '15SceneData')
num_classes = 15

# If you have a good Nvidia GPU with an appropriate environment, 
# try setting the use_GPU flag to True (the environment provided does
# not support GPUs and we will not provide any support for GPU
# computation in this project). Please note that 
# we will evaluate your implementations only using CPU mode so even if
# you use a GPU, make sure your code runs in the CPU mode with the
# environment we provided. 
use_GPU = False
if use_GPU:
    from utils_gpu import *

To train a network in PyTorch, we need 4 components:
1. **Dataset** - an object which can load the data and labels given an index.
2. **Model** - an object that contains the network architecture definition.
3. **Loss function** - a function that measures how far the network output is from the ground truth label.
4. **Optimizer** - an object that optimizes the network parameters to reduce the loss value.

This project has two main parts. In Part 1, you will train a deep network from scratch. In Part 2, you will "fine-tune" a trained network. 

## Part 0. Warm up! Training a Deep Network from Scratch

In [9]:
# Fix random seeds so that results will be reproducible
set_seed(0, use_GPU)

You do not need to code anything for this part. You will simply run the code we provided, but we want you to report the result you got. This section will also familiarize you with the steps of training a deep network from scratch. 

In [10]:
# Training parameters.
input_size = (64, 64)
RGB = False  
base_lr = 1e-2  # may try a smaller lr if not using batch norm
weight_decay = 5e-4
momentum = 0.9

We will first create our datasets, by calling the create_datasets function from student_code. This function returns a separate dataset loader for each split of the dataset (training and testing/validation). Each dataloader is used to load the datasets after appling some pre-processing transforms. In Part 1, you will be asked to add a few more pre-processing transforms to the dataloaders by modifying this function.

In [16]:
# Create the training and testing datasets.
train_dataset, test_dataset = sc.create_datasets(data_path=data_path, input_size=input_size, rgb=RGB)
assert test_dataset.classes == train_dataset.classes

Computing pixel mean and stdev...
Batch 0 / 30
Batch 20 / 30
Done, mean = 
[0.45579668]
std = 
[0.23624939]
Computing pixel mean and stdev...
Batch 0 / 60
Batch 20 / 60
Batch 40 / 60
Done, mean = 
[0.45517009]
std = 
[0.2350788]


Now we will create our network model using the SimpleNet class from student_code. The implementation provided in the SimpleNet class gives you a basic network. In Part 1, you will be asked to add a few more layers to this network. 

In [17]:
# Create the network model.
model = sc.SimpleNet(num_classes=num_classes, rgb=False, verbose=False)
if use_GPU:
    model = model.cuda()
print(model)

SimpleNet(
  (features): Sequential(
    (0): Conv2d(1, 15, kernel_size=(9, 9), stride=(1, 1), bias=False)
    (1): MaxPool2d(kernel_size=7, stride=7, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (classifier): Conv2d(15, 15, kernel_size=(8, 8), stride=(1, 1))
)


Next we will create the loss function and the optimizer. 

In [18]:
# Create the loss function.
# see http://pytorch.org/docs/0.3.0/nn.html#loss-functions for a list of available loss functions
loss_function = nn.CrossEntropyLoss()

In [19]:
# Create the optimizer and a learning rate scheduler
optimizer = optim.SGD(params=model.parameters(), lr=base_lr, weight_decay=weight_decay, momentum=momentum)
# Currently a simple step scheduler.
# See http://pytorch.org/docs/0.3.0/optim.html#how-to-adjust-learning-rate for various LR schedulers
# and how to use them
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=60, gamma=0.1)

Finally we are ready to train our network! We will start a local server to see the training progress of our network. Open a new terminal and activate the environment for this project. Then run the following command: **python -m visdom.server**. This will start a local server. The terminal output should give out a link like: "http://localhost:8097". Open this link in your browser. After you run the following block, visit this link again, and you will be able to see graphs showing the progress of your training! If you do not see any graphs, select Part 1 on the top left bar where is says Environment (only select Part 1, do not check main or Part 2).

In [20]:
params = {'n_epochs': 100, 'batch_size': 50, 'experiment': 'part1'}
trainer = Trainer(train_dataset, test_dataset, model, loss_function, optimizer, lr_scheduler, params)
best_prec1 = trainer.train_val()
print('Best top-1 Accuracy = {:4.3f}'.format(best_prec1))

---------------------------------------
Experiment: part1
n_epochs: 100
shuffle: True
checkpoint_file: None
do_val: True
num_workers: 4
batch_size: 50
experiment: part1
val_freq: 1
print_freq: 100
resume_optim: True
---------------------------------------
part1 Epoch 0 / 100
train part1: batch 0/29, loss 2.715, top-1 accuracy 4.000, top-5 accuracy 36.000
train part1: loss 2.642281
val part1: batch 0/59, loss 1.745, top-1 accuracy 58.000, top-5 accuracy 80.000
val part1: loss 2.592395
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 1 / 100
train part1: batch 0/29, loss 2.446, top-1 accuracy 16.000, top-5 accuracy 68.000
train part1: loss 2.438984
val part1: batch 0/59, loss 2.741, top-1 accuracy 6.000, top-5 accuracy 42.000
val part1: loss 2.374634
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 2 / 100
train part1: batch 0/29, loss 2.385, top-1 accuracy 22.000, top-5 accuracy 62.000
train part1: loss 2.184192
val part1: batch 0/59, loss 2.685, top-1 accuracy 4.000, top-5 

val part1: loss 1.862778
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 31 / 100
train part1: batch 0/29, loss 1.486, top-1 accuracy 58.000, top-5 accuracy 88.000
train part1: loss 1.640777
val part1: batch 0/59, loss 2.031, top-1 accuracy 38.000, top-5 accuracy 86.000
val part1: loss 2.116181
Checkpoint saved
part1 Epoch 32 / 100
train part1: batch 0/29, loss 1.447, top-1 accuracy 48.000, top-5 accuracy 94.000
train part1: loss 1.792708
val part1: batch 0/59, loss 5.552, top-1 accuracy 2.000, top-5 accuracy 14.000
val part1: loss 2.026216
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 33 / 100
train part1: batch 0/29, loss 1.567, top-1 accuracy 58.000, top-5 accuracy 84.000
train part1: loss 1.768202
val part1: batch 0/59, loss 3.677, top-1 accuracy 16.000, top-5 accuracy 56.000
val part1: loss 2.116181
Checkpoint saved
part1 Epoch 34 / 100
train part1: batch 0/29, loss 1.834, top-1 accuracy 52.000, top-5 accuracy 84.000
train part1: loss 1.616732
val part1: batch 0/59

val part1: loss 1.699665
Checkpoint saved
part1 Epoch 63 / 100
train part1: batch 0/29, loss 1.100, top-1 accuracy 62.000, top-5 accuracy 94.000
train part1: loss 1.070126
val part1: batch 0/59, loss 3.065, top-1 accuracy 24.000, top-5 accuracy 60.000
val part1: loss 1.660352
Checkpoint saved
part1 Epoch 64 / 100
train part1: batch 0/29, loss 1.178, top-1 accuracy 72.000, top-5 accuracy 92.000
train part1: loss 1.067461
val part1: batch 0/59, loss 2.683, top-1 accuracy 26.000, top-5 accuracy 70.000
val part1: loss 1.603413
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 65 / 100
train part1: batch 0/29, loss 0.929, top-1 accuracy 66.000, top-5 accuracy 98.000
train part1: loss 1.054227
val part1: batch 0/59, loss 2.425, top-1 accuracy 32.000, top-5 accuracy 74.000
val part1: loss 1.608229
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 66 / 100
train part1: batch 0/29, loss 1.090, top-1 accuracy 62.000, top-5 accuracy 96.000
train part1: loss 1.072260
val part1: batch 0/5

val part1: loss 1.613884
Checkpoint saved
part1 Epoch 95 / 100
train part1: batch 0/29, loss 0.888, top-1 accuracy 68.000, top-5 accuracy 98.000
train part1: loss 1.014711
val part1: batch 0/59, loss 2.601, top-1 accuracy 28.000, top-5 accuracy 72.000
val part1: loss 1.669646
Checkpoint saved
part1 Epoch 96 / 100
train part1: batch 0/29, loss 0.929, top-1 accuracy 76.000, top-5 accuracy 94.000
train part1: loss 0.990027
val part1: batch 0/59, loss 3.060, top-1 accuracy 26.000, top-5 accuracy 68.000
val part1: loss 1.613739
Checkpoint saved
part1 Epoch 97 / 100
train part1: batch 0/29, loss 0.782, top-1 accuracy 78.000, top-5 accuracy 98.000
train part1: loss 0.969085
val part1: batch 0/59, loss 2.865, top-1 accuracy 26.000, top-5 accuracy 72.000
val part1: loss 1.652037
Checkpoint saved
part1 Epoch 98 / 100
train part1: batch 0/29, loss 1.057, top-1 accuracy 66.000, top-5 accuracy 92.000
train part1: loss 1.004526
val part1: batch 0/59, loss 2.345, top-1 accuracy 32.000, top-5 accuracy

Expect this code to take around 5 minutes on CPU or 3 minutes on GPU. Now you are ready to actually modify the functions we used to train our model. Before you move on, make sure to record the accuracy of your network from Part 0, and report it in your write up. 

## Part 1: Modifying the Dataloaders and the Simple Network create_datasets

In [21]:
# Fix random seeds so that results will be reproducible
set_seed(0, use_GPU)

Now you will modify the create_datasets function from student_code. You will add random left-right mirroring and normalization to the transformations applied to the training dataset. You will also add normalization to the transformations applied to the testing dataset. 

In [33]:
# Create the training and testing datasets.
train_dataset, test_dataset = sc.create_datasets(data_path=data_path, input_size=input_size, rgb=RGB)
assert test_dataset.classes == train_dataset.classes

Computing pixel mean and stdev...
Batch 0 / 30
Batch 20 / 30
Done, mean = 
[0.45579668]
std = 
[0.23624939]
Computing pixel mean and stdev...
Batch 0 / 60
Batch 20 / 60
Batch 40 / 60
Done, mean = 
[0.45517009]
std = 
[0.2350788]


Now you will modify SimpleNet by adding droppout, batch normalization, and additional convolution/maxpool/relu layers. You should achieve an accuracy of at least **50%**. Make sure your network passes this threshold--it is required for full credit on this section!

You can also use the following two blocks to determine the stucture of your network.

In [38]:
# create the network model
model = sc.SimpleNet(num_classes=num_classes, rgb=False, verbose=False)
if use_GPU:
    model = model.cuda()
print(model)

SimpleNet(
  (features): Sequential(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
    (1): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
    (4): Conv2d(20, 30, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (5): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): Dropout(p=0.5)
  )
  (classifier): Conv2d(30, 15, kernel_size=(5, 5), stride=(1, 1))
)


In [39]:
# Use this block to determine the kernel size of the conv2d layer in the classifier
# first, set the kernel size of that conv2d layer to 1, and run this block
# then, use that size of input to the classifier printed by this block to
# go back and update the kernel size of the conv2d layer in the classifier
# Finally, run this block again and verify that the network output size is a scalar
# Don't forget to re-run the block above every time you update the SimpleNet class!
from torch.autograd import Variable
data, _ = train_dataset[0]
s = data.size()
data = Variable(data.view(1, *s))
if use_GPU:
    data = data.cuda()
out = model(data)
print('Network output size is ', out.size())

Network output size is  torch.Size([15])


Next we will create the loss function and the optimizer. You do not have to modify the custom_part1_trainer in student_code if you use the same loss_function, optimizer, scheduler and parameters (n_epoch, batch_size etc.) as provided in this notebook to hit the required threshold of 50% accuracy. If you changed any of these values, it is important that you modify this function in student_code since we will not be using the notebook you submit to evaluate. 

In [40]:
# Set up the trainer. You can modify custom_part1_trainer in
# student_copy.py if you want to try different learning settings.
custom_part1_trainer = sc.custom_part1_trainer(model)

if custom_part1_trainer is None:
    # Create the loss function.
    # see http://pytorch.org/docs/0.3.0/nn.html#loss-functions for a list of available loss functions
    loss_function = nn.CrossEntropyLoss()

    # Create the optimizer and a learning rate scheduler.
    optimizer = optim.SGD(params=model.parameters(), lr=base_lr, weight_decay=weight_decay, momentum=momentum)
    # Currently a simple step scheduler, but you can get creative.
    # See http://pytorch.org/docs/0.3.0/optim.html#how-to-adjust-learning-rate for various LR schedulers
    # and how to use them
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=60, gamma=0.1)

    params = {'n_epochs': 100, 'batch_size': 50, 'experiment': 'part1'}
    
else:
    if 'loss_function' in custom_part1_trainer:
        loss_function = custom_part1_trainer['loss_function']
    if 'optimizer' in custom_part1_trainer:
        optimizer = custom_part1_trainer['optimizer']
    if 'lr_scheduler' in custom_part1_trainer:
        lr_scheduler = custom_part1_trainer['lr_scheduler']
    if 'params' in custom_part1_trainer:
        params = custom_part1_trainer['params']

We are ready to train our network! As before, we will start a local server to see the training progress of our network (if you server is already running, you should not start another one). Open a new terminal and activate the environment for this project. Then run the following command: **python -m visdom.server**. This will start a local server. The terminal output should give out a link like: "http://localhost:8097". Open this link in your browser. After you run the following block, visit this link again, and you will be able to see graphs showing the progress of your training! If you do not see any graphs, select Part 1 on the top left bar where is says Environment (only select Part 1, do not check main or Part 2).

In [41]:
# Train the network!
start_time = timeit.default_timer()

trainer = Trainer(train_dataset, test_dataset, model, loss_function, optimizer, lr_scheduler, params)
best_prec1 = trainer.train_val()
print('Best top-1 Accuracy = {:4.3f}'.format(best_prec1))

print(timeit.default_timer() - start_time)

---------------------------------------
Experiment: part1
n_epochs: 100
shuffle: True
checkpoint_file: None
do_val: True
num_workers: 4
batch_size: 50
experiment: part1
val_freq: 1
print_freq: 100
resume_optim: True
---------------------------------------
part1 Epoch 0 / 100
train part1: batch 0/29, loss 2.737, top-1 accuracy 4.000, top-5 accuracy 38.000
train part1: loss 2.465942
val part1: batch 0/59, loss 1.470, top-1 accuracy 56.000, top-5 accuracy 84.000
val part1: loss 2.432343
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 1 / 100
train part1: batch 0/29, loss 2.270, top-1 accuracy 26.000, top-5 accuracy 70.000
train part1: loss 2.157839
val part1: batch 0/59, loss 3.318, top-1 accuracy 2.000, top-5 accuracy 50.000
val part1: loss 2.108668
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 2 / 100
train part1: batch 0/29, loss 2.045, top-1 accuracy 32.000, top-5 accuracy 74.000
train part1: loss 1.899284
val part1: batch 0/59, loss 2.244, top-1 accuracy 20.000, top-5

train part1: loss 0.895539
val part1: batch 0/59, loss 1.866, top-1 accuracy 28.000, top-5 accuracy 84.000
val part1: loss 1.222167
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 31 / 100
train part1: batch 0/29, loss 0.854, top-1 accuracy 72.000, top-5 accuracy 100.000
train part1: loss 0.868749
val part1: batch 0/59, loss 1.656, top-1 accuracy 40.000, top-5 accuracy 86.000
val part1: loss 1.268759
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 32 / 100
train part1: batch 0/29, loss 0.711, top-1 accuracy 76.000, top-5 accuracy 96.000
train part1: loss 0.927699
val part1: batch 0/59, loss 1.840, top-1 accuracy 38.000, top-5 accuracy 88.000
val part1: loss 1.253776
Checkpoint saved
part1 Epoch 33 / 100
train part1: batch 0/29, loss 0.759, top-1 accuracy 74.000, top-5 accuracy 98.000
train part1: loss 0.885086
val part1: batch 0/59, loss 1.553, top-1 accuracy 42.000, top-5 accuracy 90.000
val part1: loss 1.188145
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part1 Epoch 34 /

train part1: batch 0/29, loss 0.596, top-1 accuracy 80.000, top-5 accuracy 96.000
train part1: loss 0.620017
val part1: batch 0/59, loss 1.737, top-1 accuracy 38.000, top-5 accuracy 90.000
val part1: loss 1.118371
Checkpoint saved
part1 Epoch 63 / 100
train part1: batch 0/29, loss 0.587, top-1 accuracy 82.000, top-5 accuracy 100.000
train part1: loss 0.587029
val part1: batch 0/59, loss 1.750, top-1 accuracy 34.000, top-5 accuracy 92.000
val part1: loss 1.129393
Checkpoint saved
part1 Epoch 64 / 100
train part1: batch 0/29, loss 0.628, top-1 accuracy 78.000, top-5 accuracy 96.000
train part1: loss 0.569282
val part1: batch 0/59, loss 1.749, top-1 accuracy 36.000, top-5 accuracy 90.000
val part1: loss 1.119104
Checkpoint saved
part1 Epoch 65 / 100
train part1: batch 0/29, loss 0.643, top-1 accuracy 82.000, top-5 accuracy 94.000
train part1: loss 0.589406
val part1: batch 0/59, loss 1.740, top-1 accuracy 32.000, top-5 accuracy 92.000
val part1: loss 1.122527
Checkpoint saved
part1 Epoch 

train part1: loss 0.528986
val part1: batch 0/59, loss 1.627, top-1 accuracy 38.000, top-5 accuracy 92.000
val part1: loss 1.119395
Checkpoint saved
part1 Epoch 95 / 100
train part1: batch 0/29, loss 0.522, top-1 accuracy 82.000, top-5 accuracy 98.000
train part1: loss 0.545434
val part1: batch 0/59, loss 1.720, top-1 accuracy 34.000, top-5 accuracy 92.000
val part1: loss 1.119458
Checkpoint saved
part1 Epoch 96 / 100
train part1: batch 0/29, loss 0.684, top-1 accuracy 74.000, top-5 accuracy 98.000
train part1: loss 0.566093
val part1: batch 0/59, loss 1.761, top-1 accuracy 34.000, top-5 accuracy 92.000
val part1: loss 1.122475
Checkpoint saved
part1 Epoch 97 / 100
train part1: batch 0/29, loss 0.465, top-1 accuracy 86.000, top-5 accuracy 100.000
train part1: loss 0.539765
val part1: batch 0/59, loss 1.756, top-1 accuracy 34.000, top-5 accuracy 92.000
val part1: loss 1.125104
Checkpoint saved
part1 Epoch 98 / 100
train part1: batch 0/29, loss 0.526, top-1 accuracy 82.000, top-5 accurac

Make sure you get at least 50% accuracy in this section! If you tried different settings than the ones provided to get 50%, you should modify custom_part1_trainer in student code to return a dictionary with your changed settings. 

## Part 2. Fine-Tuning a Pre-Trained Network

In [296]:
# Fix random seeds so that results will be reproducible
set_seed(0, use_GPU)

Training a network from scratch takes a lof of time. Instead of training from scratch, we can take a pre-trained model and fine tune it for our purposes. This is the goal of Part 2--you will train a pre-trained network, and achieve at least 80% accuracy. 

In [297]:
# training parameters
input_size = (224, 224)
RGB = True
base_lr = 1e-3
weight_decay = 5e-4
momentum = 0.9
backprop_depth = 3

In [298]:
# Create the training and testing datasets.
train_dataset, test_dataset = sc.create_datasets(data_path=data_path, input_size=input_size, rgb=RGB)
assert test_dataset.classes == train_dataset.classes

Computing pixel mean and stdev...
Batch 0 / 30
Batch 20 / 30
Done, mean = 
[0.45611589 0.45611589 0.45611589]
std = 
[0.24786406 0.24786406 0.24786406]
Computing pixel mean and stdev...
Batch 0 / 60
Batch 20 / 60
Batch 40 / 60
Done, mean = 
[0.45549639 0.45549639 0.45549639]
std = 
[0.24698076 0.24698076 0.24698076]


Following block loads a pretrained AlexNet.

In [340]:
# Create the network model.
model = alexnet(pretrained=True)
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

Now, you modify create_part2_model from student code in order to fine-tune AlexNet. As you can see in the docs (https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py) and in the model printout above, AlexNet has 2 parts: 'features', which constists of conv layers that extract feature maps from the image, and 'classifier' which consists of FC layers that classify the features. We want to replace the last Linear layer in model.classifier. 

In [341]:
model = sc.create_part2_model(model, num_classes)
if use_GPU:
    model = model.cuda()
print(model)

from torch.autograd import Variable
data, _ = train_dataset[0]
s = data.size()
data = Variable(data.view(1, *s))
if use_GPU:
    data = data.cuda()
out = model(data)
print('Network output size is ', out.size())

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

Next we will create the loss function and the optimizer. Just as with part 1, if you modify any of the setttings to hit the required accuracy, you must modify custom_part2_trainer function to return a dictionary containing your changes. 

In [342]:
# Set up the trainer. You can modify custom_part2_trainer in
# student_copy.py if you want to try different learning settings.
custom_part2_trainer = sc.custom_part2_trainer(model)

if custom_part2_trainer is None:
    # Create the loss function
    # see http://pytorch.org/docs/0.3.0/nn.html#loss-functions for a list of available loss functions
    loss_function = nn.CrossEntropyLoss()

    # Since we do not want to optimize the whole network, we must extract a list of parameters of interest that will be
    # optimized by the optimizer.
    params_to_optimize = []

    # List of modules in the network
    mods = list(model.features.children()) + list(model.classifier.children())

    # Extract parameters from the last `backprop_depth` modules in the network and collect them in
    # the params_to_optimize list.
    for m in mods[::-1][:backprop_depth]:
        params_to_optimize.extend(list(m.parameters()))

    # Construct the optimizer    
    optimizer = optim.SGD(params=params_to_optimize, lr=base_lr, weight_decay=weight_decay, momentum=momentum)

    # Create a scheduler, currently a simple step scheduler, but you can get creative.
    # See http://pytorch.org/docs/0.3.0/optim.html#how-to-adjust-learning-rate for various LR schedulers
    # and how to use them
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    params = {'n_epochs': 4, 'batch_size': 10, 'experiment': 'part2'} 
    
else:
    if 'loss_function' in custom_part2_trainer:
        loss_function = custom_part2_trainer['loss_function']
    if 'optimizer' in custom_part2_trainer:
        optimizer = custom_part2_trainer['optimizer']
    if 'lr_scheduler' in custom_part2_trainer:
        lr_scheduler = custom_part2_trainer['lr_scheduler']
    if 'params' in custom_part2_trainer:
        params = custom_part2_trainer['params']

We are ready to fine tune our network! Just like before, we will start a local server to see the training progress of our network. Open a new terminal and activate the environment for this project. Then run the following command: **python -m visdom.server**. This will start a local server. The terminal output should give out a link like: "http://localhost:8097". Open this link in your browser. After you run the following block, visit this link again, and you will be able to see graphs showing the progress of your training! If you do not see any graphs, select Part 2 on the top left bar where is says Environment (only select Part 2, do not check main or Part 1).

In [343]:
# Train the network!
trainer = Trainer(train_dataset, test_dataset, model, loss_function, optimizer, lr_scheduler, params)
best_prec1 = trainer.train_val() 
print('Best top-1 Accuracy = {:4.3f}'.format(best_prec1))

---------------------------------------
Experiment: part2
n_epochs: 4
num_workers: 4
checkpoint_file: None
val_freq: 1
do_val: True
print_freq: 100
resume_optim: True
batch_size: 10
experiment: part2
shuffle: True
---------------------------------------
part2 Epoch 0 / 4
train part2: batch 0/149, loss 2.687, top-1 accuracy 10.000, top-5 accuracy 60.000
train part2: batch 100/149, loss 0.484, top-1 accuracy 80.000, top-5 accuracy 100.000
train part2: loss 0.926721
val part2: batch 0/298, loss 2.404, top-1 accuracy 20.000, top-5 accuracy 100.000
val part2: batch 100/298, loss 1.471, top-1 accuracy 40.000, top-5 accuracy 100.000
val part2: batch 200/298, loss 1.442, top-1 accuracy 50.000, top-5 accuracy 100.000
val part2: loss 0.600752
Checkpoint saved
BEST TOP1 ACCURACY SO FAR
part2 Epoch 1 / 4
train part2: batch 0/149, loss 0.815, top-1 accuracy 70.000, top-5 accuracy 100.000
train part2: batch 100/149, loss 0.502, top-1 accuracy 90.000, top-5 accuracy 100.000
train part2: loss 0.384019

Expect this code to take around 10 minutes on CPU or 30 seconds on GPU. You should hit 80% accuracy. 